In [1]:
import mechanicalsoup as ms
from urlpath import URL
from ratelimiter  import RateLimiter
import time
import json

In [42]:
import urlpath

In [43]:
urlpath.__version__

'1.2.0'

In [45]:
URL("http://google.com").with_query({"dd":tuple([1,2,3,4])})

URL('http://google.com?dd=1&dd=2&dd=3&dd=4')

In [2]:
from sqlalchemy import Column, Integer, String, ForeignKey, Boolean, UniqueConstraint

In [3]:
from sqlalchemy.orm import declarative_base, relationship
Base = declarative_base()

In [4]:
!rm capture.db

In [5]:
class User(Base):
    __tablename__ = "user"
    id = Column(Integer, primary_key=True)
    json = Column(String)

    
class Forum(Base):
    __tablename__ = "forum"
    id = Column(Integer, primary_key=True)
    url = Column(String)
    categories_crawled = Column(Boolean, unique=False, default=False)
    categories = relationship("Category" , back_populates="forum", cascade="all, delete-orphan")
    
class Category(Base):
    __tablename__ = "category"
    id = Column(Integer, primary_key=True)
    category_id = Column(Integer)
    forum_id = Column(Integer, ForeignKey('forum.id'))
    topic_url = Column(String)
    json = Column(String)
    pages_crawled = Column(Boolean, unique=False, default=False)
    forum = relationship("Forum", back_populates="categories")
    pages = relationship ("Page" , back_populates="category", cascade="all, delete-orphan")
    topics = relationship ("Topic" , back_populates="category", cascade="all, delete-orphan")
    __table_args__ = (UniqueConstraint('category_id', 'forum_id', name='alternate_key_cat_forum'),
                     )
class Page(Base):
    __tablename__ = "page"
    id = Column(Integer, primary_key=True)
    page_id = Column(Integer)
    category_id = Column(Integer, ForeignKey('category.id'))
    more_topics_url = Column(String)
    json = Column(String)
    category = relationship("Category", back_populates="pages")
    __table_args__ = (UniqueConstraint('category_id', 'page_id', name='alternate_key_page_cat'),)
    

In [6]:

class Topic(Base):
    __tablename__ = "topic"
    id = Column(Integer, primary_key=True)
    topic_id = Column(Integer)
    category_id = Column(Integer, ForeignKey('category.id'))
    page_excerpt_json = Column(String)
    topic_json = Column(String)
    category = relationship("Category", back_populates="topics")
    posts_crawled = Column(Boolean, unique=False, default=False)
    posts = relationship ("Post" , back_populates="topic", cascade="all, delete-orphan")
    __table_args__ = (UniqueConstraint('category_id', 'topic_id', name='alternate_key_topic_cat'),)
    
class Post(Base):
    __tablename__ = "post"
    id = Column(Integer, primary_key=True)
    post_id = Column(Integer)
    topic_id = Column(Integer, ForeignKey('topic.id'))
    json = Column(String)
    topic = relationship("Topic", back_populates="posts")
    __table_args__ = (UniqueConstraint('topic_id', 'post_id', name='alternate_key_post_topic'),)
    

In [7]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///capture.db", echo=True, future=True)

In [8]:
Base.metadata.create_all(engine)

2023-01-11 12:15:28,585 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-11 12:15:28,586 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2023-01-11 12:15:28,586 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-11 12:15:28,587 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2023-01-11 12:15:28,587 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-11 12:15:28,587 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("forum")
2023-01-11 12:15:28,587 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-11 12:15:28,588 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("forum")
2023-01-11 12:15:28,588 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-11 12:15:28,589 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("category")
2023-01-11 12:15:28,589 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-11 12:15:28,589 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("category")
2023-01-11 12:15:28,589 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01

In [62]:
class RLBrowser(ms.StatefulBrowser):
    def __init__(self, rl:RateLimiter, *args, **kwargs):
        ms.StatefulBrowser.__init__(self, *args, **kwargs)
        self.rl = rl
        
    def get(self, *args, **kwargs):
        with self.rl:
            return ms.StatefulBrowser.get(self, *args, **kwargs)

def limited(until):
    duration = int(round(until - time.time()))
    print('Rate limited, sleeping for {:d} seconds'.format(duration))

rate_limiter = RateLimiter(max_calls=3, period=1, callback=limited)
browser = RLBrowser(rate_limiter)

In [63]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind = engine)
session = Session()
base_url = URL("https://forum.inaturalist.org")
forum = Forum(url=str(base_url))
session.add(forum)
session.commit()
forum.id

2023-01-11 16:42:57,217 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-11 16:42:57,217 INFO sqlalchemy.engine.Engine INSERT INTO forum (url, categories_crawled) VALUES (?, ?)
2023-01-11 16:42:57,218 INFO sqlalchemy.engine.Engine [cached since 1.605e+04s ago] ('https://forum.inaturalist.org', 0)
2023-01-11 16:42:57,218 INFO sqlalchemy.engine.Engine COMMIT
2023-01-11 16:42:57,222 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-11 16:42:57,222 INFO sqlalchemy.engine.Engine SELECT forum.id AS forum_id, forum.url AS forum_url, forum.categories_crawled AS forum_categories_crawled 
FROM forum 
WHERE forum.id = ?
2023-01-11 16:42:57,223 INFO sqlalchemy.engine.Engine [cached since 1.605e+04s ago] (2,)


2

In [64]:
session.rollback()
query = select(Page)\
            .where(Page.category_id==1)\
            .order_by(Page.page_id.desc())
res = session.execute(query)
print(res.first())

2023-01-11 16:43:01,318 INFO sqlalchemy.engine.Engine ROLLBACK
2023-01-11 16:43:01,319 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-11 16:43:01,320 INFO sqlalchemy.engine.Engine SELECT page.id, page.page_id, page.category_id, page.more_topics_url, page.json 
FROM page 
WHERE page.category_id = ? ORDER BY page.page_id DESC
2023-01-11 16:43:01,320 INFO sqlalchemy.engine.Engine [cached since 1.605e+04s ago] (1,)
(<__main__.Page object at 0x7f86dd0382b0>,)


In [65]:
x=[1,2]
x[20:]

[]

## First we crawl the categories

In [66]:
from sqlalchemy import select

def create_posts(t:Topic, json_posts, session):
    print(json_posts)
    posts = json_posts["post_stream"]["posts"]
    for post in posts:
        p = Post(post_id=post["id"], topic_id=t.id, json=json.dumps(post))
        session.add(p)
    return len(posts)

def crawl_topic(t:Topic, base_url, browser, session):
    print("Crawling topic ", t.topic_id)
    if not t.posts_crawled:
        url = (base_url/("/t/"+str(t.topic_id)+".json"))
        resp = browser.get(url)
        t.topic_json = resp.content
        json_topic = json.loads(resp.content)
        n_posts = create_posts(t, json_topic, session)
        remaining_posts = json_topic["post_stream"]["stream"][n_posts:]
        while len(remaining_posts) > 0:
            next_posts = remaining_posts[:20]
            print(next_posts)
            url = (base_url/("/t/"+str(t.topic_id)+"/posts.json"))\
                .with_query({"post_ids[]": tuple(next_posts), "include_suggested":"true"})
            print("URL:",url)
            resp = browser.get(url)
            print(resp)
            print(resp.content)
            json_posts = json.loads(resp.content)
            n_posts = create_posts(t, json_posts, session)
            remaining_posts = remaining_posts[n_posts:]
        t.posts_crawled = True
        session.commit()
        
def crawl_category(c:Category, base_url, browser, session):
    print("Crawling category ", c.category_id)
    if not c.pages_crawled:
        query = select(Page)\
            .where(Page.category_id==c.id)\
            .order_by(Page.page_id.desc())
        last_page = session.execute(query).first()
        if last_page is not None:
            last_page = last_page[0]
        print(last_page)
        #print(last_page.keys())
        #i = 0
        while (last_page is None) or (last_page.more_topics_url is not None):
            #if i>3:
            #    break
            #else:
            #    i+=1
            if last_page is None:
                url = (base_url/("/c/"+str(c.category_id)+".json")).with_query({"page":0})
                next_page_id = 0
            else:
                next_page_id = last_page.page_id + 1
                more_url = last_page.more_topics_url.replace("?",".json?")
                url = (base_url/more_url)
            print(url)
            resp = browser.get(url)
            json_page = json.loads(resp.content)
            #print(json_page)
            topic_list = json_page["topic_list"]
            more_topics_url = topic_list["more_topics_url"] if "more_topics_url" in topic_list else None
            p = Page(category_id=c.id, 
                     page_id=next_page_id, 
                     more_topics_url=more_topics_url, 
                     json=resp.content)
            session.add(p)
            this_page_topics = []
            for topic in topic_list["topics"]:
                t = Topic(topic_id=topic["id"], category_id=c.id, page_excerpt_json=json.dumps(topic))
                this_page_topics.append(t)
                session.add(t)
            session.commit()
            for t in this_page_topics:
                crawl_topic(t, base_url, browser, session)
            last_page = p
        
def crawl_forum(forum_id, base_url, browser, session):
    f = session.get(Forum, forum_id)
    if not f.categories_crawled:
        resp = browser.get(base_url/"categories.json")
        category_list = json.loads(resp.content)["category_list"]
        print(category_list.keys())
        categories = category_list["categories"]
        for cat in categories:
            c = Category(category_id = cat["id"], forum_id=f.id, topic_url=cat["topic_url"], json=json.dumps(cat))
            session.add(c)
        f.categories_crawled = True
        session.commit()
    for c in f.categories:
        crawl_category(c, base_url, browser, session)
        
        print(c.category_id)
        #q = session.execute(select(Category).where(Category.forum_id==))
        #for category in session.get(Categories):
        

In [67]:
crawl_forum(1, base_url, browser, session)

2023-01-11 16:43:06,283 INFO sqlalchemy.engine.Engine SELECT forum.id AS forum_id, forum.url AS forum_url, forum.categories_crawled AS forum_categories_crawled 
FROM forum 
WHERE forum.id = ?
2023-01-11 16:43:06,284 INFO sqlalchemy.engine.Engine [generated in 0.00106s] (1,)
2023-01-11 16:43:06,285 INFO sqlalchemy.engine.Engine SELECT category.id AS category_id_1, category.category_id AS category_category_id, category.forum_id AS category_forum_id, category.topic_url AS category_topic_url, category.json AS category_json, category.pages_crawled AS category_pages_crawled 
FROM category 
WHERE ? = category.forum_id
2023-01-11 16:43:06,285 INFO sqlalchemy.engine.Engine [cached since 1.606e+04s ago] (1,)
Crawling category  14
2023-01-11 16:43:06,286 INFO sqlalchemy.engine.Engine SELECT page.id, page.page_id, page.category_id, page.more_topics_url, page.json 
FROM page 
WHERE page.category_id = ? ORDER BY page.page_id DESC
2023-01-11 16:43:06,286 INFO sqlalchemy.engine.Engine [cached since 1.

2023-01-11 16:43:06,983 INFO sqlalchemy.engine.Engine COMMIT
2023-01-11 16:43:06,991 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-11 16:43:06,992 INFO sqlalchemy.engine.Engine SELECT topic.id AS topic_id_1, topic.topic_id AS topic_topic_id, topic.category_id AS topic_category_id, topic.page_excerpt_json AS topic_page_excerpt_json, topic.topic_json AS topic_topic_json, topic.posts_crawled AS topic_posts_crawled 
FROM topic 
WHERE topic.id = ?
2023-01-11 16:43:06,993 INFO sqlalchemy.engine.Engine [cached since 6224s ago] (2376,)
Crawling topic  32333
{'post_stream': {'posts': [{'id': 199378, 'name': 'Katswm', 'username': 'katswm', 'avatar_template': '/user_avatar/forum.inaturalist.org/katswm/{size}/27071_2.png', 'created_at': '2022-05-30T18:46:42.781Z', 'cooked': '<p>Hi! New here… really enjoying the site.</p>\n<p>I was wondering how I should handle a photo that has more than on species in it that I would like confirmation on the IDs. Thanks!</p>', 'post_number': 1, 'post_type'

2023-01-11 16:43:07,300 INFO sqlalchemy.engine.Engine [cached since 6015s ago] (b'{"post_stream":{"posts":[{"id":199378,"name":"Katswm","username":"katswm","avatar_template":"/user_avatar/forum.inaturalist.org/katswm/{size}/27071_ ... (42418 characters truncated) ... t","internal":false,"attachment":false,"reflection":false,"clicks":2,"user_id":863,"domain":"www.inaturalist.org","root_domain":"inaturalist.org"}]}}', 1, 2376)
2023-01-11 16:43:07,301 INFO sqlalchemy.engine.Engine INSERT INTO post (post_id, topic_id, json) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-01-11 16:43:07,301 INFO sqlalchemy.engine.Engine [cached since 6015s ago (insertmanyvalues)] (199378, 2376, '{"id": 199378, "name": "Katswm", "username": "katswm", "avatar_template": "/user_avatar/forum.inaturalist.org/katswm/{size}/27071_2.png", "created_at ... (1080 characters trunc

2023-01-11 16:43:07,554 INFO sqlalchemy.engine.Engine [cached since 6015s ago] (b'{"post_stream":{"posts":[{"id":198338,"name":"tallard","username":"tallard","avatar_template":"/user_avatar/forum.inaturalist.org/tallard/{size}/119 ... (50743 characters truncated) ... t","internal":false,"attachment":false,"reflection":false,"clicks":1,"user_id":887,"domain":"www.inaturalist.org","root_domain":"inaturalist.org"}]}}', 1, 2377)
2023-01-11 16:43:07,555 INFO sqlalchemy.engine.Engine INSERT INTO post (post_id, topic_id, json) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-01-11 16:43:07,555 INFO sqlalchemy.engine.Engine [cached since 6015s ago (insertmanyvalues)] (198338, 2377, '{"id": 198338, "name": "tallard", "username": "tallard", "avatar_template": "/user_avatar/forum.inaturalist.org/tallard/{size}/11947_2.png", "created ... (1146 characters truncated) ... "mentioned_u

<Response [200]>
b'{"post_stream":{"posts":[{"id":198493,"name":"Diana Studer","username":"dianastuder","avatar_template":"/user_avatar/forum.inaturalist.org/dianastuder/{size}/2769_2.png","created_at":"2022-05-24T12:39:42.518Z","cooked":"\\u003caside class=\\"quote no-group\\" data-username=\\"pdfuenteb\\" data-post=\\"17\\" data-topic=\\"32162\\"\\u003e\\n\\u003cdiv class=\\"title\\"\\u003e\\n\\u003cdiv class=\\"quote-controls\\"\\u003e\\u003c/div\\u003e\\n\\u003cimg loading=\\"lazy\\" alt=\\"\\" width=\\"20\\" height=\\"20\\" src=\\"https://sea2.discourse-cdn.com/business6/user_avatar/forum.inaturalist.org/pdfuenteb/40/5015_2.png\\" class=\\"avatar\\"\\u003e pdfuenteb:\\u003c/div\\u003e\\n\\u003cblockquote\\u003e\\n\\u003cp\\u003ephenology of some obs. didn\xe2\x80\x99t match totally\\u003c/p\\u003e\\n\\u003c/blockquote\\u003e\\n\\u003c/aside\\u003e\\n\\u003cp\\u003eDoesn\xe2\x80\x99t matter whether they are deliberately cheating or not - they are devaluing iNat data. Should be \xe2

...

2023-01-11 16:43:45,934 INFO sqlalchemy.engine.Engine [cached since 6053s ago] (b'{"post_stream":{"posts":[{"id":197270,"name":"Jason Hernandez","username":"jasonhernandez74","avatar_template":"/user_avatar/forum.inaturalist.org/j ... (32204 characters truncated) ... tem","avatar_template":"https://global.discourse-cdn.com/business6/uploads/inaturalist/original/2X/7/7831452cf7f0d653ff1ffd34176334b7ef1b6a17.png"}}}', 1, 2442)
2023-01-11 16:43:45,935 INFO sqlalchemy.engine.Engine INSERT INTO post (post_id, topic_id, json) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-01-11 16:43:45,935 INFO sqlalchemy.engine.Engine [cached since 6053s ago (insertmanyvalues)] (197270, 2442, '{"id": 197270, "name": "Jason Hernandez", "username": "jasonhernandez74", "avatar_template": "/user_avatar/forum.inaturalist.org/jasonhernandez74/{si ... (4774 characters truncated) ... "mentioned_users": [], "calendar_details": [], "can_acce

2023-01-11 16:43:46,335 INFO sqlalchemy.engine.Engine [cached since 6054s ago] (b'{"post_stream":{"posts":[{"id":197362,"name":"Marcia Morris","username":"dull2shinetoo","avatar_template":"/user_avatar/forum.inaturalist.org/dull2s ... (23202 characters truncated) ... st","internal":false,"attachment":false,"reflection":false,"clicks":1,"user_id":20,"domain":"www.inaturalist.org","root_domain":"inaturalist.org"}]}}', 1, 2443)
2023-01-11 16:43:46,336 INFO sqlalchemy.engine.Engine INSERT INTO post (post_id, topic_id, json) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-01-11 16:43:46,336 INFO sqlalchemy.engine.Engine [cached since 6054s ago (insertmanyvalues)] (197362, 2443, '{"id": 197362, "name": "Marcia Morris", "username": "dull2shinetoo", "avatar_template": "/user_avatar/forum.inaturalist.org/dull2shinetoo/{size}/1512 ... (2075 characters truncated) ... "mentioned_users": [], "calendar_details": [], "can_accept_answer": false, "ca

2023-01-11 16:43:46,582 INFO sqlalchemy.engine.Engine [cached since 6054s ago] (b'{"post_stream":{"posts":[{"id":196871,"name":"Sam Hambly","username":"hamsambly","avatar_template":"/user_avatar/forum.inaturalist.org/hamsambly/{si ... (46081 characters truncated) ... tem","avatar_template":"https://global.discourse-cdn.com/business6/uploads/inaturalist/original/2X/7/7831452cf7f0d653ff1ffd34176334b7ef1b6a17.png"}}}', 1, 2444)
2023-01-11 16:43:46,583 INFO sqlalchemy.engine.Engine INSERT INTO post (post_id, topic_id, json) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-01-11 16:43:46,583 INFO sqlalchemy.engine.Engine [cached since 6054s ago (insertmanyvalues)] (196871, 2444, '{"id": 196871, "name": "Sam Hambly", "username": "hamsambly", "avatar_template": "/user_avatar/forum.inaturalist.org/hamsambly/{size}/25388_2.p

2023-01-11 16:43:47,158 INFO sqlalchemy.engine.Engine [cached since 6054s ago] (b'{"post_stream":{"posts":[{"id":197284,"name":"","username":"Mr_Fizzy","avatar_template":"https://avatars.discourse-cdn.com/v4/letter/m/958977/{size} ... (15630 characters truncated) ... tem","avatar_template":"https://global.discourse-cdn.com/business6/uploads/inaturalist/original/2X/7/7831452cf7f0d653ff1ffd34176334b7ef1b6a17.png"}}}', 1, 2445)
2023-01-11 16:43:47,159 INFO sqlalchemy.engine.Engine INSERT INTO post (post_id, topic_id, json) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-01-11 16:43:47,159 INFO sqlalchemy.engine.Engine [cached since 6054s ago (insertmanyvalues)] (197284, 2445, '{"id": 197284, "name": "", "username": "Mr_Fizzy", "avatar_template": "https://avatars.discourse-cdn.com/v4/letter/m/958977/{size}.png", "created_at" ... (1220 characters truncated) ... "mentioned_users": [], "calendar_details": [], "can_accept_answer": false, "can_unaccept_answer": false, "accep

KeyboardInterrupt: 

In [13]:


q = session.execute(select(Forum))
for x in q:
    print(x[0].id)


2023-01-11 12:15:43,582 INFO sqlalchemy.engine.Engine SELECT forum.id, forum.url, forum.categories_crawled 
FROM forum
2023-01-11 12:15:43,583 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ()
1


In [14]:
result = session.query(Category).where(Category.id==3)
print(result)
for c in result:
    print(c.id)

SELECT category.id AS category_id_1, category.category_id AS category_category_id, category.forum_id AS category_forum_id, category.topic_url AS category_topic_url, category.json AS category_json, category.pages_crawled AS category_pages_crawled 
FROM category 
WHERE category.id = ?
2023-01-11 12:15:43,587 INFO sqlalchemy.engine.Engine SELECT category.id AS category_id_1, category.category_id AS category_category_id, category.forum_id AS category_forum_id, category.topic_url AS category_topic_url, category.json AS category_json, category.pages_crawled AS category_pages_crawled 
FROM category 
WHERE category.id = ?
2023-01-11 12:15:43,588 INFO sqlalchemy.engine.Engine [generated in 0.00067s] (3,)
3


In [15]:
def crawl_category(category_id, )

SyntaxError: expected ':' (3839712522.py, line 1)

In [ ]:
def get_category_page(category_url, page):
    page_category_url = category_url.with_query({"page":page})
    resp = browser.get(first_page_category_url)
    print(resp)
    print(resp.content)
    return json.loads(resp.content)

def crawl_category_topics(category_url):
    ids = []
    i = 0
    has_more = True
    while has_more:
        json_resp = get_category_page(category_url, i)
        print(json_resp.keys())
        break
        topic_list = json_resp["topic_list"]
        ids.extend([topic["id"] for topic in topic_list["topics"]])
        i += 1
        print(i)
        has_more = ("more_topics_url" in topic_list)
        


for link in browser.links("^/c/"):
    ref = link.attrs["href"]
    category_url = base_url/(ref+".json")
    topic_ids = get_topic_ids(category_url)
    break
    

In [ ]:
json_resp = json.loads(resp.content)

In [ ]:
json_resp.keys()

In [ ]:
topic_list = json_resp["topic_list"]
print("more_topics_url" in topic_list)

In [ ]:
ids = [topic["id"] for topic in topic_list["topics"]]

In [ ]:
print(ids)

In [ ]:
for link in browser.links("/t/"):
    print(link)

In [ ]:
for link in browser.page.select('a'):
    print(link.text, '->', link.attrs['href'])